# 配置YOLOv4适用的本地Linux环境

---

## 主要软件配置

- Ubuntu 18.04
- Opencv 3.4.10
- CUDA 10.1.105
- Cudnn 7.6.5

---

### Ubuntu

于官网下载对应版本系统`ISO`镜像文件:[18.04](https://ubuntu.com/download/alternative-downloads)。<br>
如使用U盘安装，可用如`UltralISO`将`ISO`镜像写入U盘。<br>
注意在启动时进入`Boot`/`Uefi`界面，开启U盘启动并切换启动盘。<br>
全新盘配置分区时需要注意分区情况：
- '\'分区：主分区，必要区域，无系统整理需求时甚至只需此一个分区即可。<br>
- '\swap'分区：交换分区，如Win10的虚拟内存，向内存IO提供缓冲服务，可与当前系统内存大小对应。<br>
- '\boot'与'\uefi'分区：启动分区，往往放在最前，用于存储管理系统内核文件，一般只需500Mb~1G，看个人需求系统环境内核大小进行定制<br>
- '\home'分区：用户文件分区，此类目录下的文件在更换内核，重装系统时可以选择完全保留。<br>

安装时有skip内容，可直接跳过，一般选择直接安装，而不在安装时更新。<br>
安装完毕后注意显卡驱动是否已经安装，执行`nvidia-smi`命令可查看。<br>
从你的应用程序菜单打开`软件和更新`（softare/updates），单击`附加驱动`选项，可以看到Nvidia卡使用的驱动程序和专有驱动程序列表，选择应用后重启生效（可在设置中查看是否生效）<br>

### OpenCV

主要参考[资料](https://blog.csdn.net/qq_35798758/article/details/107911794?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control)来源。<br>
安装前注意系统版本与`/etc/apt/source.list`下指定库来源版本一致，否则会导致冲突。<br>
注意安装库时可能会失败，具体情况根据终端中返回信息进行处理，部分可选库的安装失败不会导致之后的YOLOv4程序无法编译，如：`无法定位软件包 libjasper-dev`，还请按个人情况处理相应问题。<br>
OpenCV可前往[Release](https://opencv.org/releases/)界面进行下载，本次选择的3.4.10版本的`Source`源码版本进行本地编译。<br>
源码`build`完毕后需要配置相关系统变量`path`才可被系统在调用时自动找到，请留意。

In [1]:
# 依赖库安装
# compiler 编译需求库
!sudo apt-get install build-essential
# required 运行需求前置依赖库
!sudo apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev
# optional 可选库
!sudo apt-get install python-dev python-numpy libtbb2 libtbb-dev libjpeg-dev libpng-dev libtiff-dev libjasper-dev libdc1394-22-dev
# 在下载完毕OpenCV对应版本源码文件后，解压文件并在终端中切入其文件目录。
%cd opencv-3.4.10
# 创建Make路径并进入
!mkdir opencv_build_dir
%cd opencv_build_dir
# 开始编译文件
!cmake -D CMAKE_BUILD_TYPE=Release-D CMAKE_INSTALL_PREFIX=/usr/local ..
# 根据CPU的核心/线程情况指定同步工作节点数量
!make -j6
!sudo make install 
# 环境变量配置
!sudo gedit /etc/ld.so.conf.d/opencv.conf
# 文件中添加如下内容：
# export /usr/local/
# 刷新环境配置
!sudo ldconfig
# 修改bashrc文件
!sudo gedit /etc/bash.bashrc
# 在文件末尾追加：
# PKG_CONFIG_PATH=$PKG_CONFIG_PATH:/usr/local/lib/pkgconfig
# export PKG_CONFIG_PATH
# 刷新bashrc配置
!source /etc/bash.bashrc
# 查看安装是否生效
!opencv_version

### CUDA & CUDNN

#### **CUDA**

本次使用的CUDA版本为[10.1.105](https://developer.nvidia.com/cuda-toolkit-archive)，可供安装方式有两种，一为`runfile`，直接运行安装后配置`path`变量即可，一为`.deb`安装包，两者根据[官方指导](https://developer.nvidia.com/cuda-10.1-download-archive-base?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=deblocal)安装配置即可。<br>
如安装过程中遇到问题参考[此篇博客](https://blog.csdn.net/qq_34296627/article/details/82833172)。<br>
同时，`runfile`版本的环境配置可以参考[官方文档](https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html#post-installation-actions)提供的相关信息。<br>


#### **CUDNN**

CUDNN本次使用版本为[7.6.5](https://developer.nvidia.com/rdp/cudnn-archive)，要注意其版本需要与CUDA版本匹配。<br>
其也有多种安装方式，最直观一种为直接利用`cuDNN Library for Linux`，当然，使用`cuDNN Runtime Library for Ubuntu18.04 (Deb)`，`cuDNN Developer Library for Ubuntu18.04 (Deb)`亦可（注意系统版本匹配）。<br>
实验中根据[官方指导](https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html#installlinux-tar)，验证了以`cuDNN Library for Linux`的`.tar`包文件进行CUDNN安装。

In [ ]:
# CUDA runfile (base installer)
# 1. Run `sudo sh cuda_10.1.105_418.39_linux.run`
# 2. Follow the command-line prompts
# CUDA .deb (base installer)
!sudo dpkg -i cuda-repo-ubuntu1804-10-1-local-10.1.105-418.39_1.0-1_amd64.deb
# 注册其public key，否则无法正常使用
!sudo apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda
# 编辑环境变量
!sudo gedit /etc/bash.bashrc
# 文件末尾添加如下内容：(此处系统版本为64位，默认选择Lib64)
# export PATH=/usr/local/cuda/bin${PATH:+:${PATH}}
# export LD_LIBRARY_PATH=/usr/local/cuda/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
# export CUDA_HOME=/usr/local/cuda
# 激活环境修改
!source /etc/bash.bashrc
# 进入CUDNN目录，解压相关文件：
!tar -xzvf cudnn-x.x-linux-x64-v8.x.x.x.tgz
# PS:solitairetheme8文件可转为tar文件进行解压
!cp  cudnn-x.x-linux-x64-v8.x.x.x.solitairetheme8 cudnn-x.x-linux-x64-v8.x.x.x.tgz
# 将CUDNN中依赖库加入对应版本CUDA目录即可，需要注意CUDA的安装位置
!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include
!sudo cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
# 权限赋予操作
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*
# 利用.deb文件进行安装（runtime与developer library）
!sudo dpkg -i libcudnn8_x.x.x-1+cudax.x_amd64.deb
!sudo dpkg -i libcudnn8-dev_8.x.x.x-1+cudax.x_amd64.deb
# 验证当前CUDA安装是否生效
!nvcc -V

---

## 其他配置

- Apt包管理镜像源更换
- Pip命令镜像源更换
- Conda命令镜像源更换
- [可参考其他资料](https://www.jianshu.com/p/9bcaadf00a3b)

---

### Apt-get更换镜像源

图形化桌面系统中，可以在：设置 -> 关于 -> Software Updates -> Ubuntu软件的下载自栏目中更换下载源。<br>
直接进行`apt`设置更改，通过编辑`'/etc/apt/sources.list'`文件实现：
- 进行`sources.list`文件备份
- 进入并编辑`sources.list`，将所有Ubuntu默认镜像地址更换到目标源地址上。
- 保存并让`apt`命令更新自身镜像库配置。
- 注意系统版本名要保持匹配，如`https://mirrors.tuna.tsinghua.edu.cn/ubuntu/ bionic main restricted universe multiverse`应应用于系统`Ubuntu 18.04 LTS bionic`中。

可使用的推荐源有：
- [清华源](https://mirrors.tuna.tsinghua.edu.cn/help/ubuntu/)：`mirrors.tuna.tsinghua.edu.cn`
- [阿里云](https://developer.aliyun.com/mirror/)：`mirrors.aliyun.com`
- [网易云](http://mirrors.163.com/.help/ubuntu.html)：`mirrors.163.com`



In [ ]:
# 备份
!sudo cp /etc/apt/sources.list /etc/apt/sources.list.bak
# 编辑
!sudo gedit /etc/apt/sources.list
# 更新apt源镜像配置
!sudo apt-get update

### Pip命令镜像源更换

`pip`命令源更换需要修改/创建`~/.pip/pip.conf`文件进行指定。`~`目录指向为当前用户的文件系统根目录。<br>
内容可以为：<br>
```
    [global]
    index-url = https://pypi.tuna.tsinghua.edu.cn/simple
```
可使用的源有：
- 阿里云：http://mirrors.aliyun.com/pypi/simple/ 
- 中国科技大学：https://pypi.mirrors.ustc.edu.cn/simple/ 
- 豆瓣：http://pypi.douban.com/simple/
- 清华大学：https://pypi.tuna.tsinghua.edu.cn/simple/
- 中国科学技术大学：http://pypi.mirrors.ustc.edu.cn/simple/


In [ ]:
# 临时使用
!pip install -i source-url some-package
# 命令行更改（需要pip版本>=10)
!pip config set global.index-url source-url
# 创建目录
!mkdir ~/.pip
# 编辑配置
!gedit ~/.pip/pip.conf
# 查看配置结果
!pip config list

### Conda命令更换镜像源

与`pip`类似，`conda`需要配置`~/.condarc`文件进行源更换。<br>
内容可以为：
```
    channels:
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/menpo/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/bioconda/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/
        - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
        - defaults
```
同时，`conda`也可以利用命令行增加/删除自定义源配置。<br>
同理，也可以配置其他源供`conda`使用。<br>
目前已知问题为几个国内源暂时不可使用，暂未找到合适解决方案，建议先不配置conda源。


In [2]:
# 添加镜像源配置
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/bioconda/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/menpo/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/
!conda config --set show_channel_urls yes
# 查看配置结果
!conda config
# 清除自定义镜像源
!conda config --remove-key channels

---

## 额外软件配置

- Anaconda：Python环境管理
- VsCode：编码控制
- Scrcpy：Linux安卓设备投屏控制

### Anaconda安装

于[Anaconda官网](https://www.anaconda.com/products/individual)选择合适的系统版本Anaconda进行下载。<br>
在终端中利用`bash`命令执行下载的`.sh`脚本文件，进行Anaconda安装。<br>
之后跟随安装指示进行操作即可，Anaconda会自动配置其相关路径，无需再手动调整。<br>
可以唤醒`Anaconda-navigator`的图形界面对当前Python环境进行编辑。


In [ ]:
# 执行脚本
!bash Anaconda~.sh
# 调起Anaconda-Navigator(当Anaconda被安装在默认路径下时)
!source ~/anaconda3/bin/activate root
!anaconda-navigator

### VsCode安装

于[VsCode官网](https://code.visualstudio.com/)下载对应安装包，Ubuntu以Deepin为基本构建，对应安装包为`.deb`格式文件<br>
下载完毕后，在其安装文件所在目录，利用`dpkg`命令进行安装。<br>
也可以利用Ubuntu自带的应用商店进行安装，但要注意下载文件位置，不能在临时文件目录下，否则无法加载。


In [ ]:
# 安装释放命令
!sudo dpkg -i vscode_amd64.deb

### Scrcpy安装

`Scrcpy`能够对打开USB调试的安卓设备进行投屏，在Linux，MacOS和Windows上操作安卓设备。<br>
可以根据[Github官方发布区](https://github.com/Genymobile/scrcpy/)上的指导进行安装。

In [ ]:
# Scrcpy安装
# apt方法，适用于20.04版本
!apt install scrcpy
# snap方法，适用于其他版本
!sudo snap install scrcpy
# 唤醒Scrcpy
!scrcpy


---

## 环境配置的便捷替代方案

- Docker：一键部署CUDA环境

### Docker安装使用

由于需要唤起GPU加速运算，需要利用Nvidia-Docker进行辅助，此处给出[官方安装指导](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#docker)，是否取用参考自身需求。<br>
主要关注在Deepin与Ubuntu上的安装指南，注意依赖安装顺序，Toolkit安装时也请注意版本匹配问题。<br>
在利用`Docker`启动`Container`时，要注意指定`Image`的CUDA版本信息。

In [ ]:
# 唤起一个Container，查看其配置信息
!docker run --gpus all nvidia/cuda:10.2-base nvidia-smi

同样的，Docker在配置国内源后能够有效的进行下载加速，具体操作如下：<br>
- 进入编辑`/etc/docker/daemon.json`
- 配置`Json`格式配置信息
```Json
        {
            "registry-mirrors":["https://3laho3y3.mirror.aliyuncs.com"],
            "runtimes":{
                        "nvidia":{
                            "path":"nvidia-container-runtime",
                            "runtimeArgs":[]
                }
            }
        }
```

In [ ]:
# 编辑指定文件
!sudo gedit /etc/docker/daemon.json

进入[Docker官网](https://hub.docker.com/)，查询自身需要的CUDA版本镜像。<br>
利用`docker pull`命令将其从服务器上拉取至本地启动。<br>
更多后续编辑操作参考[此博文](https://blog.csdn.net/a454193977/article/details/106383605/)。

In [ ]:
# 拉取Image
!sudo docker pull nvidia/cuda:10.2-cudnn7-devel-ubuntu18.04

---

## YOLOv4的不同版本编译运行

- CPU：开启OpenCV版本编译
- GPU：完整YOLOv4网络编译
- TIPS：所遇部分问题

### CPU版本构建

Linux环境下，YOLOv4作者在`darknet`文件夹下给出`build.sh`脚本进行编译，在未配置CUDA环境但OpenCV已经正确配置的前提下，会自动编译构建CPU训练版本的YOLOv4网络结构。<br>
若利用`make`命令，则需要手动设置开启OpenCV命令，标记当前系统中已经正确配置OpenCV，并执行`make`完成编译。

In [ ]:
# 用此命令查看当前系统中是否有配置CUDA
!nvcc -V
# 在未配置CUDA与CUDNN前，利用cmake自动构建CPU版本YOLOv4网络框架
%cd ./darknet/
!./build.sh
# 利用make配置编译CPU版本YOLOv4网络
%cd ./darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!make

### GPU版本构建

同理，Linux环境下，配置完毕CUDA与CUDNN后，利用`build.sh`可直接编译GPU版本YOLOv4。<br>
`make`命令部分也同理。

In [ ]:
# 此时CUDA与CUDNN配置已完毕，nvcc -V命令能够正常响应
%cd ./darknet/
!./build.sh
# 利用make编译GPU版本YOLOv4
%cd ./darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# 注意CUDNN_HALF的开启对显卡算力有具体要求
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

### 所遇问题

- 'build.sh'脚本无法执行：文件每行结尾处符号可能出错，注意Linux下脚本文件每行结束无换行符。
- 经尝试，16系列显卡下，`CUDA 11.1`版本，在编译YOLOv4网络成功后，加载权重扫描图片无结果输出，CPU-OpenCV版本正常。
- 经尝试，16系列显卡下，`CUDA 10.1`与`CUDNN 7.6.5`版本，GPU-OpenCV版本输出正常生效。

---

## YOLOv4训练指令（部分）
*详细训练内容指导将放入另一篇Notebook中，此处主要为简单的模型构建有效性验证*

In [ ]:
# 放入一张图片，利用已经训练完毕的权重文件进行测试。在cpu与gpu版本中输出无差别即为构建成功
!./darknet detector test cfg/coco.data cfg/yolov4.cfg ../YOLOv4-Config/yolov4.weights ./data/dog.jpg